# Class 15:  joint entropy and the REVEAL algorithm

We'll use the bladder cancer gene expression data to test out the REVEAL algorithm. First, we'll load the data and filter to include only genes for which the median log2 expression level is > 14.  That should give us 13 genes to work with (to keep running times reasonable for an in-class notebook). We'll use a "dynamic programming"-type approach in which we store the joint entropy of *G*+*X* at stage *i-1* of the algorithm, and use that stored joint entropy to get
the joint entropy of *X* at stage *i*.

Import the Python modules that we will need for this exercise

In [ ]:
import pandas as pd
import numpy as np
import itertools
import pprint

Download the file https://csx46.s3-us-west-2.amazonaws.com/bladder_cancer_genes_tcga.txt to the local file `bladder_cancer_genes_tcga.txt`

In [ ]:
!curl https://csx46.s3-us-west-2.amazonaws.com/bladder_cancer_genes_tcga.txt > bladder_cancer_genes_tcga.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29.9M  100 29.9M    0     0  24.9M      0  0:00:01  0:00:01 --:--:-- 24.9M


Load the data file `bladder_cancer_genes_tcga.txt` into a `pandas.DataFrame`, convert it to a `numpy.ndarray` matrix, and print the matrix dimensions

In [ ]:
gene_matrix_for_network_df = pd.read_csv("bladder_cancer_genes_tcga.txt", sep="\t")
gene_matrix_for_network = gene_matrix_for_network_df.values
print(gene_matrix_for_network.shape)

(4473, 414)


Filter the matrix to include only rows for which the column-wise median is > 14; matrix should now be 13 x 414 (print the shape in order to check it).

In [ ]:
genes_keep = np.where(np.median(gene_matrix_for_network, axis=1) > 14)
matrix_filt = gene_matrix_for_network[genes_keep, ][0]
print(matrix_filt.shape)
N, M = matrix_filt.shape

(13, 414)


Binarize the gene expression matrix using the mean value as a breakpoint, turning it into a NxM matrix of booleans (`True`/`False`).  Call it `gene_matrix_binarized`.   Use `numpy.tile` and `numpy.mean` and `transpose`.

In [ ]:
gene_matrix_binarized = np.tile(np.mean(matrix_filt, axis=1),(M,1)).transpose() < matrix_filt
print(gene_matrix_binarized.shape)

(13, 414)


Print the first four columns of the first four rows of your matrix, as a sanity check.

In [ ]:
gene_matrix_binarized[0:4, 0:4]

array([[False,  True, False, False],
       [False,  True, False, False],
       [ True,  True,  True, False],
       [False,  True, False, False]])

The core part of the REVEAL algorithm is a function that can compute the joint entropy of a collection of binary (TRUE/FALSE) vectors X1, X2, ..., Xn (where length(X1) = length(Xi) = M).
Write a function `entropy_multiple_vecs` that takes as its input a nxM matrix (where n is the number of variables, i.e., genes, and M is the number of samples in which gene expression was measured). The function should use the log2 definition of the Shannon entropy. It should return the joint entropy H(X1, X2, ..., Xn) as a scalar numeric value. I have created a skeleton version of this function for you, in which you can fill in the code. I have also created some test code that you can use to test your function, below.

In [ ]:
def entropy_multiple_vecs(binary_vecs: np.array):
    ## use shape to get the numbers of rows and columns as [n,M]
    [n, M] = binary_vecs.shape

    # make a "M x n" dataframe from the transpose of the matrix binary_vecs
    binary_df = pd.DataFrame(binary_vecs.transpose())

    # use the groupby method to obtain a data frame of counts of unique occurrences of the 2^n possible logical states
    binary_df_counts = binary_df.groupby(binary_df.columns.values.tolist()).size().values

    # divide the matrix of counts by M, to get a probability matrix
    probvec = binary_df_counts/M

    # compute the shannon entropy using the formula
    hvec = -probvec*np.log2(probvec)
    return np.sum(hvec)

This test case should produce the value 3.47:

In [ ]:
print(f"{entropy_multiple_vecs(gene_matrix_binarized[0:4,]):0.2f}")

3.47


## Example implementation of the REVEAL algorithm:
In this exercise, we will implement the REVEAL algorithm up to Stage 3.

Define a constant `ratio_thresh` that will control the threshold for the conditional entropy of a gene given its candidate regulator(s); if that conditional entropy is below `ratio_thresh` times the

In [ ]:
ratio_thresh = 0.1

Initialize a list of length *N*, called `regulators`. Initialize all entries of the list to `None`. As the algorithm progresses, this will become a list-of-sets that gives the regulators for each gene. So, the final state of this variable is basically the output of the REVEAL algorithm.

In [ ]:
regulators = [None]*N

We will control the limit on the number of stages of the REVEAL algorithm that will run, using an integer constant `max_stage`; set it to 3.

In [ ]:
max_stage = 3

Create a list `entropies_for_stages` of length *N*, initialized with `None` values. Go through all the stage values from 0 to `max_stage-1` (inclusive), and in each case, create a numpy array with shape `[N]*(stage + 1)`
and initialize all entries in the numpy array to `np.nan`.
Save that array in entry `stage` in `entropies_for_stages`.

In [ ]:
entropies_for_stages = [None]*N

# for `stage` in `0...(max_stage)`: (inclusive)
for stage in range(0, max_stage + 1):
    # define a variable `shape` with value `[N]*(stage + 1)`
    shape = [N]*(stage + 1)

    # make a numpy array `stage_entropies` of shape `shape` and fill it with `np.nan`
    stage_entropies = np.empty(shape)
    stage_entropies[:] = np.nan

    # save `stage_entropies` as entry `stage` in `entropies_for_stages`
    entropies_for_stages[stage] = stage_entropies

    print(f"for entry {stage}, the array shape is: {stage_entropies.shape}")

for entry 0, the array shape is: (13,)
for entry 1, the array shape is: (13, 13)
for entry 2, the array shape is: (13, 13, 13)
for entry 3, the array shape is: (13, 13, 13, 13)


Loop over all genes, and for each integer gene index *i*, extract the slice `[i,:,None]` from `gene_matrix_binarized` (which extracts an array of shape `(414,1)`), take its transpose, and compute `entropy_multiple_vecs` on it. Use the resulting value to fill entry `i` of `entropies_for_stages[0]`.  After that loop is complete, print `entropies_for_stages[0]` to sanity-check how it looks.

In [ ]:
for i in range(0,N):
    single_row_matrix = gene_matrix_binarized[i,:,None].transpose()
    entropies_for_stages[0][i] = entropy_multiple_vecs(single_row_matrix)
entropies_for_stages[0]

array([0.98579539, 0.95033767, 0.99892231, 0.99957909, 0.94459118,
       0.99256314, 0.99715307, 0.99973063, 0.99973063, 0.99939387,
       0.99107606, 0.99998317, 0.99715307])

Create a set `all_genes` of length *N* and initialize it to the values 1..*N-1*.

In [ ]:
all_genes = set(range(0, N))

Create a set `genes_to_fit`, that will initially contain all the values 0,...,*N-1*. Each time the algorithm discovers the regulators for some gene *i*, element *i* will be removed from this set, as the algorithm proceeds. Here, we are just initializing the set.

In [ ]:
genes_to_fit = set(range(0,N))

Now, let's implement REVEAL. It starts out quite easily: just loop over stages. Then there is an inner loop over genes in the set `genes_to_fit`. Within that loop, things get more... interesting. See the comments inline.

In [ ]:
# for `stage` in `1...(max_stage + 1)`:
for stage in range(1, max_stage + 1):
    print(f"starting stage {stage}..., wish me luck")
    # we will be removing items from `genes_to_fit` inside this loop, so iterate
    # on a copy of the set (for all elements of `genes_to_fit.copy()` as `gene`):
    for gene in genes_to_fit.copy():
        print(f"trying to find regulators for gene {gene}")
        # get the marginal entropy for gene `gene`; this is just a lookup
        # from entry `gene` in the array `entropies_for_stages[0]`.
        HG = entropies_for_stages[0][gene]

        # construct a set `poss_regs` of possible regulators for `gene`, as the
        # set difference of `all_genes` and a singleton set `{gene}`.
        poss_regs = all_genes - {gene}

        # create a list `poss_regs_combs` of all possible combinations of `stage`
        # genes from the set `poss_regs`, using `itertools.combinations`.
        poss_regs_combs = [list(x) for x in itertools.combinations(poss_regs, stage)]

        # create a numpy array of length `len(poss_regs_combs)` called `HX`;
        # initialize its values to `np.nan`
        HX = np.empty(len(poss_regs_combs))
        HX[:] = np.nan

        # create a numpy array of length `len(poss_regs_combs)` called `HGX`;
        # initialize its values to `np.nan`
        HGX = np.empty(len(poss_regs_combs))
        HGX[:] = np.nan

        for i in range(0, len(poss_regs_combs)):
            # get element `i` from `poss_regs_combs`, and assign it to `poss_regs_comb`
            poss_regs_comb = poss_regs_combs[i]

            # index into the numpy array `entropies_for_stages[stage - 1]`
            # using tuple(sorted(poss_regs_comb)), and assign to `HXval`
            HXval = entropies_for_stages[stage - 1][tuple(sorted(poss_regs_comb))]

            # if HXval is `np.nan`, compute it from scratch by calling
            # `entropy_multiple_vecs` on the result of indexing
            # `gene_matrix_binarized` with `[poss_regs_comb, :]`:
            if np.isnan(HXval):
                HXval = entropy_multiple_vecs(gene_matrix_binarized[poss_regs_comb,:])

            # store `HXval` in position `i` in `HX`
            HX[i] = HXval

            # make a sorted list `regs` from `[gene] + poss_regs_comb`
            regs = sorted([gene] + poss_regs_comb)

            # compute the joint entropy on `gene_matrix_binarized[regs, :]`
            # and assign it to `HGXval`
            HGXval = entropy_multiple_vecs(gene_matrix_binarized[regs, :])

            # store `HGXval` at position `i` in `HGX`
            HGX[i] = HGXval

            # store `HGXval` at index `tuple(regs)` in the numpy array
            # `entropies_for_stages[stage]`, if the array contains `np.nan`
            # at that position currently:
            if np.isnan(entropies_for_stages[stage][tuple(regs)]):
                entropies_for_stages[stage][tuple(regs)] = HGXval

        # compute `np.min` of the array `HGX - HX` and store as `min_value`
        min_value = np.min(HGX - HX)

        # if `HG - min_value` is greater than or equal to `ratio_thresh * HG`:
        if HG - min_value >= ratio_thresh * HG:
            # get the index of the minimum value of `HGX-HX` as `min_pos`
            min_pos = np.argmin(HGX - HX)

            # assign the value `poss_regs_combs[min_pos]` to `regulators[gene]`.
            regulators[gene] = poss_regs_combs[min_pos]

            print(f"success! the regulators for gene {gene} are {regulators[gene]}")

            # remove `gene` from `genes_to_fit`
            genes_to_fit.remove(gene)



starting stage 1..., wish me luck
trying to find regulators for gene 0
success! the regulators for gene 0 are [1]
trying to find regulators for gene 1
success! the regulators for gene 1 are [0]
trying to find regulators for gene 2
success! the regulators for gene 2 are [8]
trying to find regulators for gene 3
trying to find regulators for gene 4
trying to find regulators for gene 5
trying to find regulators for gene 6
success! the regulators for gene 6 are [7]
trying to find regulators for gene 7
success! the regulators for gene 7 are [6]
trying to find regulators for gene 8
success! the regulators for gene 8 are [2]
trying to find regulators for gene 9
trying to find regulators for gene 10
success! the regulators for gene 10 are [11]
trying to find regulators for gene 11
success! the regulators for gene 11 are [10]
trying to find regulators for gene 12
starting stage 2..., wish me luck
trying to find regulators for gene 3
trying to find regulators for gene 4
trying to find regulators 

Pretty-print (using `pprint.pprint`) `list(enumerate(regulators))` to see the results

In [ ]:
pprint.pprint(list(enumerate(regulators)))

[(0, [1]),
 (1, [0]),
 (2, [8]),
 (3, None),
 (4, None),
 (5, None),
 (6, [7]),
 (7, [6]),
 (8, [2]),
 (9, [7, 8, 10]),
 (10, [11]),
 (11, [10]),
 (12, [1, 6, 9])]
